# [NTDS'18] milestone 2: network models
[ntds'18]: https://github.com/mdeff/ntds_2018

[Hermina Petric Maretic](https://people.epfl.ch/hermina.petricmaretic), [EPFL LTS4](https://lts4.epfl.ch)

## Students

* Team: `<your team number>`
* Students: `<the name of all students in the team>`
* Dataset: `<the dataset you used to complete the milestone>`

## Rules

* Milestones have to be completed by teams. No collaboration between teams is allowed.
* Textual answers shall be short. Typically one to two sentences.
* Code has to be clean.
* In the first part, you cannot import any other library than we imported. In the second part, you are allowed to import any library you want.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart & Run All" in Jupyter.

## Objective

The purpose of this milestone is to explore various random network models, analyse their properties and compare them to your network. In the first part of the milestone you will implement two random graph models and try to fit them to your network. In this part you are not allowed to use any additional package. In the second part of the milestone you will choose a third random graph model that you think shares some properties with your network. You will be allowed to use additional packages to construct this network, but you must explain your network choice. Finally, make your code as clean as possible, and keep your textual answers short.

## Part 0

Import the adjacency matrix of your graph that you constructed in milestone 1, as well as the number of nodes and edges of your network.

In [ ]:
import numpy as np

# We have several binary matrices created using different thresholds
filenames = ["adjacency_0.1", "adjacency_0.3", "adjacency_0.5", "adjacency_0.7", "adjacency_0.9"]

adjacencies_binary =  [np.load(file+".npy") for file in filenames]
n_nodes =  [a.shape[0] for a in adjacencies_binary]
n_edges =  [np.sum(a)/2 for a in adjacencies_binary]

## Part 1

**For the computation of this part of the milestone you are only allowed to use the packages that have been imported in the cell below.**

In [ ]:
%matplotlib inline

import random

import pandas as pd
import matplotlib.pyplot as plt
import scipy

### Question 1

Create a function that constructs an Erdős–Rényi graph.

In [ ]:
def erdos_renyi(n, p, seed=None):
    """Create an instance from the Erdos-Renyi graph model.
    
    Parameters
    ----------
    n: int
        Size of the graph.
    p: float
        Edge probability. A number between 0 and 1.
    seed: int (optional)
        Seed for the random number generator. To get reproducible results.
    
    Returns
    -------
    adjacency
        The adjacency matrix of a graph.
    """
    
    np.random.seed(seed)
    G = np.zeros((n,n))
    
    for i in range(n):
        for j in range(i,n):
            if i != j:
                r = random.random()
                if r <= p:
                    # Probability p that nodes are connected
                    G[i,j] = 1
                else:
                    continue
            else:
                # Nodes are not connected with themselves, leave the zeros
                continue
    adjacency = G + np.transpose(G)
    
    return adjacency

In [ ]:
er = erdos_renyi(5, 0.6, 9765)
plt.spy(er)
plt.title('Erdos-Renyi (5, 0.6)')

In [ ]:
er = erdos_renyi(10, 0.4, 7648)
plt.spy(er)
plt.title('Erdos-Renyi (10, 0.4)')

### Question 2

Use the function to create a random Erdos-Renyi graph. Choose the parameters such that number of nodes is the same as in your graph, and the number of edges similar. You don't need to set the random seed. Comment on your choice of parameters.

In [ ]:
# Take probability based on maximum possible number of edges and observed edges in binary adjacency with thres. = 0.5
p_link = n_edges[2]/(n_nodes[2]*(n_nodes[2]-1)/2)
er_CN = erdos_renyi(n_nodes[2], p_link)
L_er = (np.count_nonzero(er_CN))/2
L = n_edges[2]

print("The number of links in the Erdos-Renyi graph with p = {0:.3f} is {1:.0f}".format(p_link, L_er))
print("The number of links in our graph is %d" %L)

**Your answer here.**

### Question 3

Create a function that constructs a Barabási-Albert graph.

In [ ]:
def barabasi_albert(n, m, m0, seed=None):
    """Create an instance from the Barabasi-Albert graph model.
    
    Parameters
    ----------
    n: int
        Size of the graph.
    m: int
        Number of edges to attach from a new node to existing nodes.
    m0: int
        Number of initial nodes
    seed: int (optional)
        Seed for the random number generator. To get reproducible results.
    
    Returns
    -------
    adjacency
        The adjacency matrix of a graph.
    """
    
    if m > m0:
        return print("The inital number of nodes is too small")
    
    np.random.seed(seed)
    
    # initalizing the graph
    G = np.zeros((n,n))
    nodes = np.arange(m0)
    

    # connecting the initial m0 nodes together
    for i in range(m0):
        G[i,i+1] = 1
    
    for j in range(m0,n):
    
        # count degrees of existing nodes
        k_i = np.sum(G[nodes, :],axis = 1)
        
        # Nodes to which no edge exists yet
        available_nodes = nodes
    
        # add m links to new node
        rand_nb = np.random.rand(m)  

        for i in range(m):
            # compute probability of connection with each available node
            p = k_i[available_nodes]/np.sum(k_i[available_nodes])
            
            # compute cumulated probability array which can be used to classifiy a generated random number
            p_cum = np.cumsum(p)
            
            target_index = min(np.argwhere(p_cum >= rand_nb[i]))
            target_node = available_nodes[target_index]
            available_nodes = np.delete(available_nodes, target_index)
    
            # We connect the selected target node
            G[j,target_node] = 1
        
        # Finally, we add the new node to our existing node list
        nodes = np.append(nodes, j)
    
    adjacency = G + np.transpose(G)
    
    
    return adjacency


In [ ]:
ba = barabasi_albert(5, 1, 2, 9087)
plt.spy(ba)
plt.title('Barabasi-Albert (5, 1)')

In [ ]:
ba = barabasi_albert(10, 2, 2, 8708)
plt.spy(ba)
plt.title('Barabasi-Albert (10, 2)')

### Question 4

Use the function to create a random Barabási-Albert graph. Choose the parameters such that number of nodes is the same as in your graph, and the number of edges similar. You don't need to set the random seed. Comment on your choice of parameters.

In [ ]:
# With a threshold of 0.5, we have 7485 edges and 200 nodes in our original network
ba_nb_links = 48
ba_init_nodes = 48

ba_CN = barabasi_albert(200, ba_nb_links, ba_init_nodes)
plt.spy(ba_CN)
plt.title('Barabasi-Albert Conseil National (10, 2)')
print ("Simulation of a BA model with %d inital nodes, %d links per new node and final number of 200 nodes" 
       % (ba_init_nodes, ba_nb_links))
n_nodes_ba_CN =  [ba_CN.shape[0]]
print ("The number of nodes with this BA model: %d " % n_nodes_ba_CN[0])
n_edges_ba_CN =  [np.sum(ba_CN)/2]
print ("The number of edges with this BA model: %d" % n_edges_ba_CN[0])


**Your answer here**

### Question 5

Compare the number of edges in all three networks (your real network, the Erdős–Rényi network, and the Barabási-Albert netowk).

In [ ]:
# Real network with treshold of 0.5
print ("The number of nodes in the real network: %d " % n_nodes[2])
print ("The number of edges in the real network: %d " % n_edges[2])


In [ ]:
# Erdos Renyi
plt.spy(er_CN)
plt.title('Erdos-Renyi (200, {0:.3f})'.format(p_link))

print ("Simulation of a ER model with 200 nodes and treshold of {0:.3f}".format(p_link))
n_nodes_er_CN =  [er_CN.shape[0]]
print ("The number of nodes with this ER model: %d " % n_nodes_er_CN[0])
n_edges_er_CN =  [np.sum(er_CN)/2]
print ("The number of edges with this ER model: %d" % n_edges_er_CN[0])

In [ ]:
# Barbasi Albert network
plt.spy(ba_CN)
plt.title('Barabasi-Albert Conseil National (10, 2)')
print ("Simulation of a BA model with %d inital nodes, %d links per new node and final number of 200 nodes" 
       % (ba_init_nodes, ba_nb_links))
n_nodes_ba_CN =  [ba_CN.shape[0]]
print ("The number of nodes with this BA model: %d " % n_nodes_ba_CN[0])
n_edges_ba_CN =  [np.sum(ba_CN)/2]
print ("The number of nodes with this BA model: %d" % n_edges_ba_CN[0])

### Question 6

Implement a function that computes the [Kullback–Leibler (KL) divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) between two probability distributions.
We'll use it to compare the degree distributions of networks.

In [ ]:
def kl_divergence(p, q):
    """Compute the KL divergence between probability distributions of degrees of two networks.
    
    Parameters
    ----------
    p: np.array
        Probability distribution of degrees of the 1st graph.
    q: np.array
        Probability distribution of degrees of the 2nd graph.
    
    Returns
    -------
    kl
        The KL divergence between the two distributions.
    """
    
    kl=np.dot(p,np.log(p/q))
    
    return kl

In [ ]:
p_test = np.array([0.2, 0.2, 0.2, 0.4])
q_test = np.array([0.3, 0.3, 0.1, 0.3])
kl_divergence(p_test, q_test)

### Question 7

Compare the degree distribution of your network to each of the two synthetic ones, in terms of KL divergence. **Hint:** Make sure you normalise your degree distributions to make them valid probability distributions.

In [ ]:
def get_degree_distibution(adjacency, bins=None, laplace_smoothing=True, set_minmax=None):
    """"This function returns the degree distribution for a graph with adjacency matrix adjacency
    In order to compute meaningful KL divergences, the degree axis is divided into intervals
    and an average degree per intervall is computed
    Returns vector of interval midpoints (degrees), probability for node to have a degree inside a certain interval,
    average and extremas of the degree distirbution"""
    """ bins: number of intervalls between k_min and k_max
        set_minmax: The minimum and maximum of the k axis can optionally be chosen. Along with the bins argument,
        the sampling points can implicitly be chosen"""
        
    k_i = np.sum(adjacency, axis=1)/2
    k_i.sort()
    k_freq, counts = np.unique(k_i, return_counts=True)
    
    if bins == None:
        return k_freq, counts/np.sum(counts), [np.mean(k_i), (min(k_i), max(k_i))]
    
    # Create sample intervalls
    if set_minmax == None:
        dk, h = np.linspace(min(k_freq), max(k_freq), bins+1, retstep=True)
    else:
        dk, h = np.linspace(set_minmax[0], set_minmax[1], bins+1, retstep=True)
    
    # Midpoints of the sampling intervals
    k_s = dk[1:] - h/2
    counts_s = []
    prev_idx = 0
    
    # Laplace smoothing to avoid zero frequencies
    if laplace_smoothing:
        alpha = 1;
        d = len(dk[1:])
    else:
        alpha = 0;
        d = 0;
        
    # Sample degree distribution
    for k in dk[1:]:
        idx_smaller = np.argwhere(k_freq <= k)
        if idx_smaller.size == 0:
            counts_s.append(alpha)
            continue
        else:
            idx = max(idx_smaller)+1
            idx = np.asscalar(idx)
            counts_s.append(np.sum(counts[prev_idx:idx])+alpha)
            prev_idx = idx
    # Normalize distribution
    counts_s = np.array(counts_s)/(sum(counts_s)+d*alpha)
    
    # Return mean, max and min of degree distribution
    stats = [np.mean(k_i), (min(k_i), max(k_i))]
    return k_s, counts_s, stats 

In [ ]:
intervals = 40
k_minmax = (1,80)
smooth = True

k, weights, stats = get_degree_distibution(adjacencies_binary[2], bins=intervals,
                                           laplace_smoothing=smooth, set_minmax=k_minmax) 
k_er, weights_er, stats_er = get_degree_distibution(er_CN, bins=intervals, 
                                                    laplace_smoothing=smooth, set_minmax=k_minmax)
k_ba, weights_ba, stats_ba = get_degree_distibution(ba_CN, bins=intervals,
                                                    laplace_smoothing=smooth, set_minmax=k_minmax)

In [ ]:
print(stats, stats_er, stats_ba)

In [ ]:
# Calculate Kullback-Leibler divergences
kl_er = kl_divergence(weights, weights_er)
kl_ba = kl_divergence(weights,weights_ba)
print("KL-Divergence for the Erdös-Renyi network: {0:.3f}".format(kl_er))
print("KL-Divergence for the Barabasi-Albert network: {0:.3f}".format(kl_ba))

### Question 8

Plot the degree distribution historgrams for all three networks. Are they consistent with the KL divergence results? Explain.

In [ ]:
fig = plt.figure(figsize=(17, 5))
ax1 = fig.add_subplot(1,3,1)
points1= ax1.bar(k, weights)
ax1.grid(True)
ax1.set_xlabel("Degree k")
ax1.set_ylabel("Probability")
ax1.set_title("Degree distribution (k-intervalls: {0})".format(intervals))
ax2 = fig.add_subplot(1,3,2)
points2 = ax2.bar(k_er, weights_er)
ax2.grid(True)
ax2.set_xlabel("Degree k")
ax2.set_ylabel("Probability")
ax2.set_title("Degree distribution Erdös-Renyi (k-intervalls: {0})".format(intervals))
ax3 = fig.add_subplot(1,3,3)
points3 = ax3.bar(k_ba, weights_ba)
ax3.grid(True)
ax3.set_xlabel("Degree k")
ax3.set_ylabel("Probability")
ax3.set_title("Degree distribution Barabasi-Albert (k-intervalls: {0})".format(intervals))
fig.text(0.52,0.48,"D_KL = {0:.3f}".format(kl_er), fontsize=13)
fig.text(0.80,0.5,"D_KL = {0:.3f}".format(kl_ba), fontsize=13)

### Question 9

Imagine you got equal degree distributions. Would that guarantee you got the same graph? Explain.

**Your answer here.**

## Part 2

**You are allowed to use any additional library here (e.g., NetworkX, PyGSP, etc.).** Be careful not to include something here and use it in part 1!

### Question 10

Choose a random network model that fits you network well. Explain your choice. 

**Hint:** Check lecture notes for different network models and their properties. Your choice should be made based on at least one property you'd expect to be similar.

**Your answer here.**

### Question 11

Explain (in short) how the chosen model works.

**Your answer here.**

### Question 12

Create a random graph from that model, such that the number of nodes is the same as in your graph.

In [ ]:
def preferential_attachment_graph(n, n_links, seed=None):
    """Based on the Barabasi-Albert graph, but without network growth. At each instance t, a link is added to a node
    out of n nodes in the network with preferential attachment"""
    
    if n_links > n*(n-1)/2:
        return print("The number of edges is larger than the maximum number of edges possible")
    
    np.random.seed(seed)
    G = np.zeros((n,n))
    nodes = np.arange(n)
    
    # Initial attachment probability (unconnected nodes have Pi(1) => equivalent to every node having one link)
    k_i = np.ones(n)
    fully_connected_nodes = []
    
    for t in range(n_links):
        rand_number = np.random.random(2)
        # Nodes that are already fully connected can be connected more
        k1 = np.delete(k_i, fully_connected_nodes)
        # Cum. probability to connect to any of the non-fully connected nodes
        p1 = np.cumsum(k1)/np.sum(k1)
        nodes_1 = np.delete(nodes, fully_connected_nodes)
        
        # Choose first node and remove node and neighbours temporarily from list
        node_1 = np.asscalar(nodes_1[min(np.argwhere(p1 >= rand_number[0]))])
        node_1_neighbours = np.nonzero(G[node_1,:])
        
        # Attachment probability for the remaining available nodes (Can't connect to itself and already connected nodes)
        excluded_nodes = np.append(node_1, node_1_neighbours)
        k2 = np.delete(k_i, excluded_nodes)
        p2 = np.cumsum(k2)/np.sum(k2)
        nodes_2 = np.delete(nodes, excluded_nodes)

        # Choose second node
        node_2 = np.asscalar(nodes_2[min(np.argwhere(p2 >= rand_number[1]))])
        # Connect them
        G[node_1, node_2] = 1
        G[node_2, node_1] = 1
        
        # Check if nodes are now fully connected
        if np.count_nonzero(G[node_1,:])>= n-1:
            fully_connected_nodes.append(node_1)
        if np.count_nonzero(G[node_2,:]) >= n-1:
            fully_connected_nodes.append(node_2)
            
        # Update degrees
        k_i[node_1] += 1
        k_i[node_2] += 1
    
    return G


graph = preferential_attachment_graph(n_nodes[2], int(n_edges[2]), seed=2)
intervals = None
k, weights, stats = get_degree_distibution(graph, bins=intervals) 

In [ ]:
plt.figure(figsize=(6, 6))
plt.bar(k, weights)
plt.grid(True)
plt.xlabel("Degree k")
plt.ylabel("Probability")
plt.title("Degree distribution (k-intervalls: {0})".format(intervals))

### Question 13

Check the properties you expected to be similar, and compare to your network.

In [ ]:
# Your code here.

Are the results what you expected? Explain.

**Your answer here.**